In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']='4'
import numpy as np
import torch
import open3d as o3d
from tqdm import tqdm

from DDM import DDM_P2P
from utils.LieAlgebra import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Define the loss functoin. Here we set up_ratio=10, K=5, std=0.05, and beta=10.

In [2]:
loss_func=DDM_P2P(10,5,0.05,True,10)

Optimize the transformation between source and target points by minimizing the distance between them.

In [3]:
n_iters=2000

src_pcd=o3d.io.read_point_cloud('demo_data/rigid_registration/src.ply')
tgt_pcd=o3d.io.read_point_cloud('demo_data/rigid_registration/tgt.ply')

src=np.array(src_pcd.points)
tgt=np.array(tgt_pcd.points)

src=torch.from_numpy(src).cuda().float()
tgt=torch.from_numpy(tgt).cuda().float()

Reconstruction=Reconstruction_point(zero_init=True).cuda()
optimizer=torch.optim.Adam(Reconstruction.parameters(),lr=2e-2)
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,n_iters,2e-4)

for _ in tqdm(range(n_iters)):
    transformed_src,_=Reconstruction(src,None)
    loss=loss_func(transformed_src.unsqueeze(0),tgt.unsqueeze(0))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

trans = Reconstruction.Transform()
transforms = np.eye(4, 4)
transforms[:3, :3] = trans[0].detach().cpu().numpy()
transforms[:3, 3] = trans[1].detach().cpu().numpy()

transforms[:3,:3]=transforms[:3,:3].T

del optimizer, Reconstruction

print('the transformation is: ')
print(transforms)

100%|██████████| 2000/2000 [00:45<00:00, 44.04it/s]

the transformation is: 
[[ 0.97161245 -0.05005142  0.23122309  0.15128468]
 [ 0.05735305  0.99804187 -0.02496088  0.20182344]
 [-0.22952099  0.03751364  0.97258055 -0.27950671]
 [ 0.          0.          0.          1.        ]]
